In [2]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg

conc_density = 150.0 #pcf

from steel_design.AISC import aisc
from concrete_design.steel_reinforcement import rebar_df as rebar

display(Latex(r"\newpage")) #pagebreak

<IPython.core.display.Latex object>

# Concrete Slab Design
## Inputs

In [3]:
# Materials
fc = 3000.0 #psi
fy = 60000.0 #psi

# Dimensions
ln = 5 * 12 #in
b = 12.0 #in
t = 8.0 #in

# Provided Steel
bar = "#4"
Ab = rebar.loc[bar, "Area"]
n = 1 #number of bars per width, b

## Strength

In [4]:
# Minimum Thickness
t_min = ln / 30
print(f"t_min (two way) = {round(t_min, 2)} in")

t_min = ln / 20
print(f"t_min (one way) = {round(t_min, 2)} in")

t_min (two way) = 2.0 in
t_min (one way) = 3.0 in


In [5]:
# Minimum Steel
Ag = b * t #in^2
As_min = 0.0018 * Ag
As_minft = As_min * 12 / b

f"As,min = {round(As_min, 3)} in^2 or {round(As_minft, 3)} in^2/ft"

'As,min = 0.173 in^2 or 0.173 in^2/ft'

In [6]:
# Provided Steel
As = n * Ab #in^2
As_ft = As * 12 / b
f"As = {round(As, 3)} in^2 or {round(As_ft, 3)} in^2/ft"

'As = 0.2 in^2 or 0.2 in^2/ft'

In [7]:
# bar spacing
s = b / n
f"bar spacing = {round(s, 2)} in"

'bar spacing = 12.0 in'

In [8]:
# Required Thickness
phi = 0.9
d = t - 1.0 #in
a = As * fy / (0.85*fc * b)
phiMn = 0.9 * As * fy * (d - a/2) / 12 / 1000
f"phiMn = {round(phiMn, 3)} k-ft"

'phiMn = 6.124 k-ft'

## Deflection